## 필요한 라이브러리

In [90]:
import pandas as pd
import numpy as np
import requests
import re
from selenium import webdriver
from bs4 import BeautifulSoup

## 팀 딕셔너리

In [104]:
team_dict = {'Team' : ['삼성', '해태', 'KIA', '롯데' ,'삼미', '청보', '태평양', '현대', 'MBC', 'LG', 'OB', '두산', '빙그레', '한화', '쌍방울', 'SK', 'SSG', '우리', '넥센', '키움', 'NC', 'KT', '타이거즈','삼청태현', 'MBC-LG', 'OB-두산', '이글스', 'SK-SSG', '히어로즈'],
            'Team_code' : [1001, 2001, 2002, 3001, 4001, 4002, 4003, 4004, 5001, 5002, 6001, 6002, 7001, 7002, 8001, 9001, 9002, 10001, 10001, 1001, 11001, 12001, 2, 4, 5, 6, 7, 9, 11]}

team_df = pd.DataFrame(team_dict)

In [112]:
def convert_dtype(value):
    try:
        # 소숫점이 없으면 int로 변환
        if '.' not in str(value):
            return int(value)
        # 소숫점이 있으면 float으로 변환
        else:
            return float(value)
    except ValueError:
        # 변환할 수 없으면 문자열로 반환
        return str(value)

def convert_dataframe_dtypes(df):
    # 데이터프레임의 각 값을 변환
    return df.applymap(convert_dtype)


def crawling():
    q_bp = int(input('[타자 정보 불러오기] : 1\n[투수 정보 불러오기] : 2\n'))

    if q_bp == 1:
        df = crawling_batter()
    else :
        df = crawling_pitcher()
        
    return df

def crawling_batter():
    sy = int(input('[시작 시즌]\n'))
    ey = int(input('[마지막 시즌]\n'))
    te_yn = int(input('[팀 선택]\n예 : 1\n아니오 : 2\n'))
    if te_yn == 1:
        te_input = input('[팀 입력] - ex) 삼성, 타이거즈, 삼청태현, MBC-LG, OB-두산, 이글스, SK-SSG, 히어로즈\n')
        te = team_dict['Team_code'][team_dict['Team'].index(te_input)]
    else : te = None
    so_input = int(input('[정렬 기준]\n출장:1\n타율:2\nwRC+:3\nWAR:4\n'))
    so = {1:'G', 2:'AVG', 3:'wRCplus', 4:'WAR'}.get(so_input, '잘못된 입력값')
    ob_input = int(input('오름차순:1\n내림차순:2\n'))
    ob = {1: 'ASC', 2: 'DESC'}.get(ob_input, '잘못된 입력값')
    reg_input = int(input('규정 : 1\n전체 : 2\n70% : 3\n50% : 4\n30% : 5\n'))
    reg = {1: 'R', 2: 'A', 3:'P70', 4:'P50', 5:'P30'}.get(reg_input, '잘못된 입력값')
    pr = int(input('[출력 수]\n'))
    
    site = f'https://statiz.sporki.com/stats/?m=main&m2=batting&m3=default&so={so}&ob={ob}&year=2024&sy={sy}&ey={ey}&te={te}&po=&lt=10100&reg={reg}&pe=&ds=&de=&we=&hr=&ha=&ct=&st=&vp=&bo=&pt=&pp=&ii=&vc=&um=&oo=&rr=&sc=&bc=&ba=&li=&as=&ae=&pl=&gc=&lr=&pr={pr}&ph=&hs=&us=&na=&ls=0&sf1=WAROff&sk1=&sv1=&sf2=WAROff&sk2=&sv2='
    response = requests.get(site)
    soup = BeautifulSoup(response.text, 'html.parser')
    tr_tags = soup.select('tr')

    i=0
    batter_stats = []
    bat_columns = ['No', 'Name', 'Season', 'Team_no', 'Pos']


    th_tags = soup.select('tr > th')

    for th_tag in th_tags[4:26]:
        bat_columns.append(re.sub(r'[^\w\s]', '', th_tag.get_text()))

    for th_tag in th_tags[29:35]:
        bat_columns.append(re.sub(r'[^\w\s]', '', th_tag.get_text()))

    bat_columns.append('WAR')
    batter_stats.append(bat_columns)

    for tr_tag in tr_tags:
        batter_stat = []
        if (i%12==0) or (i%12==1):
            i+=1
            continue
        no = tr_tag.select_one('a')['href'].split('p_no=')[1]
        name = tr_tag.select_one('div').get_text()

        span_tag = tr_tag.select('div.teams > span')
        season = span_tag[0].get_text()
        team_no = span_tag[1].select_one('img')['src'].split('/')[-1].split('.')[0]
        position = span_tag[2].get_text()

        batter_stat = [no, name, season, team_no, position]

        td_tags = tr_tag.select('td')[4:]

        for td_tag in td_tags:
            batter_stat.append(td_tag.get_text())


        batter_stats.append(batter_stat)

        i+=1
    
    df = pd.DataFrame(batter_stats[1:], columns=batter_stats[0])
    df = convert_dataframe_dtypes(df)
    df = pd.merge(df, team_df, left_on='Team_no', right_on='Team_code', how='left').drop(columns=['Team_no', 'Team_code'])
    return df

def crawling_pitcher():
    sy = int(input('[시작 시즌] - ex)2022\n'))
    ey = int(input('[마지막 시즌] - ex)2023\n'))
    te_yn = int(input('[팀 선택]\n예 : 1\n아니오 : 2\n'))
    if te_yn == 1:
        te_input = input('[팀 입력] - ex) 삼성, 타이거즈, 삼청태현, MBC-LG, OB-두산, 이글스, SK-SSG, 히어로즈\n')
        te = team_dict['Team_code'][team_dict['Team'].index(te_input)]
    else : te = None
    so_input = int(input('[정렬 기준]\n출장:1\n이닝:2\nWAR:3\n'))
    so = {1:'G', 2:'IP', 3:'WAR'}.get(so_input, '잘못된 입력값')
    ob_input = int(input('오름차순:1\n내림차순:2\n'))
    ob = {1: 'ASC', 2: 'DESC'}.get(ob_input, '잘못된 입력값')
    reg_input = int(input('규정 : 1\n전체 : 2\n'))
    reg = {1: 'R', 2: 'A'}.get(reg_input, '잘못된 입력값')
    pr = int(input('[출력 수]\n'))

    site = f'https://statiz.sporki.com/stats/?m=main&m2=pitching&m3=default&so={so}&ob={ob}&year=2024&sy={sy}&ey={ey}&te={te}&po=&lt=10100&reg={reg}&pe=&ds=&de=&we=&hr=&ha=&ct=&st=&vp=&bo=&pt=&pp=&ii=&vc=&um=&oo=&rr=&sc=&bc=&ba=&li=&as=&ae=&pl=&gc=&lr=&pr={pr}&ph=&hs=&us=&na=&ls=&sf1=&sk1=&sv1=&sf2=&sk2=&sv2='

    response = requests.get(site)
    soup = BeautifulSoup(response.text, 'html.parser')
    tr_tags = soup.select('tr')

    i=0
    pitcher_stats = []
    pit_columns = ['No', 'Name', 'Season', 'Team_no', 'Pos']

    th_tags = soup.select('tr > th')

    for th_tag in th_tags[4:37]:
        pit_columns.append(re.sub(r'[^\w\s]', '', th_tag.get_text()))

    pitcher_stats.append(pit_columns)

    for tr_tag in tr_tags:
        pitcher_stat = []
        if (i%12==0) or (i%12==1):
            i+=1
            continue

        no = tr_tag.select_one('a')['href'].split('p_no=')[1]
        name = tr_tag.select_one('a').get_text()

        span_tag = tr_tag.select('div.teams > span')
        season = span_tag[0].get_text()
        team_no = span_tag[1].select_one('img')['src'].split('/')[-1].split('.')[0]
        position = span_tag[2].get_text()

        pitcher_stat = [no, name, season, team_no, position]

        td_tags = tr_tag.select('td')
        for td_tag in td_tags[4:]:
            pitcher_stat.append(td_tag.get_text())

        pitcher_stats.append(pitcher_stat)

        i+=1

    df = pd.DataFrame(pitcher_stats[1:], columns=pitcher_stats[0])
    df = convert_dataframe_dtypes(df)
    df = pd.merge(df, team_df, left_on='Team_no', right_on='Team_code', how='left').drop(columns=['Team_no', 'Team_code'])
    return df